# Behavioral Design Patterns


Behavioral patterns don't really follow any central theme:

- They are all different.
- Sometimes they overlap in their function, i.e., the goal they achieve, but the underlying mechanisms are different.

Table of contents:

- [Chain of Responsibility](#chain-of-responsibility)
  - Example: Support
  - Example: Card Game
  - Example: Broker and Queries
- [Command](#Command)
  - Example: Bank Account with Undo
  - Example: Bank Account with Composite Command = Macro
- [Interpreter](#Interpreter)
- [Iterator](#Iterator)
- [Mediator](#Mediator)
  - Example: Chatroom
  - Example: Game with Events
- [Memento](#Memento)
  - Example: Bank Account
  - Example: Bank Account with Undo and Redo
- [Observer](#Observer)
  - Events
  - Property Observers
  - Property Dependencies
- [State (Machine)](#state-machine)
  - Handmade State Machine
  - Switch-Based State Machine
- [Strategy](#Strategy)
  - Example: Text Processor
- [Template Method](#Template-Method)
  - Example: Chess Game
- [Visitor](#Visitor)

## Chain of Responsibility

Software components are often built with hierarchical relationships. Those relationships can capture resposibility, similarly as it can happen in a company (CEO, manager, employee). The pattern Chain of Responsibility is a chain of components in which all get a change to process a command or a query, optionally having a default processing implementation and an ability to terminate the processing chain.

In other words, Chain of Responsibility allows an object to pass a request along a chain of potential handlers until the request is handled. This pattern decouples the sender of the request from the receiver by giving multiple objects a chance to handle the request.
 
For example:

- In a GUI, we click a graphical element on a form
  - The button handles the action, which leads to stopping the processing
  - Or the underlying group box can handle it
  - Or the underlying window can handle it
- A card computer game
  - A creature has attack and defense skills
  - Thos skills can be boosted by other card creatures
- Support
  - We can have first-level or basic support
  - Or second-level = advanced support
  - Or third-level = expert support

Some key components that often appear in Chain of Responsibility are:

- **Handler**: Defines an interface for handling requests and for setting the next handler in the chain.
- **ConcreteHandler**: Implements the Handler interface and processes requests it is responsible for. If it can't handle the request, it forwards the request to the next handler in the chain.
- **Client**: Initiates the request and passes it to the first handler in the chain.

**Command and Query Separation (CQS)**: Whenever we operate on objects, we separate all the invokations into two types:

- Command: invokation which asks for an action or a change; e.g., set value to X.
- Query: ask/retrieve information without changing anything, e.g., get value of X.

Separating both Commands and Queries is known as the Command and Query Separation (CQS) Principle. It is a design priciple with some benefits:

- Clarity: Separating commands and queries makes it clear which methods are performing actions and which are retrieving information.
- Simpler Testing: Queries are easier to test because they are idempotent and do not change the state.
- Improved Maintainability: Clear separation of responsibilities makes the code easier to understand and maintain.

### Example: Support Chain

In [1]:
# Abstract Handler:
# Defines an interface for handling requests 
# and for setting the next handler in the chain.
# SupportHandler will be inherited by concrete handlers!
class SupportHandler:
    def __init__(self):
        # The next_handler is a pointer to the next handler
        # which is set with set_next_handler
        # when setting the Chain of Responsibility
        self.next_handler = None

    # This method is used to set the
    # Chain of Responsibility
    def set_next_handler(self, handler):
        self.next_handler = handler

    # This handle_request method is called by the concrete
    # class/object if the type of issue to handle 
    # does not belong to the type of the concrete handler
    def handle_request(self, issue):
        if self.next_handler:
            return self.next_handler.handle_request(issue)
        else:
            return "No handler available for this issue"

In [2]:
# Concrete Handlers:
# They implement the Handler interface 
# and processes requests it is responsible for. 
# If they can't handle the request, 
# they forward the request to the next handler in the chain.
class BasicSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "basic":
            return "BasicSupport: Handling basic issue"
        else:
            # We call the abstract handler
            # which calls the next handler
            # in the Chain of Responsibility
            return super().handle_request(issue)

class AdvancedSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "advanced":
            return "AdvancedSupport: Handling advanced issue"
        else:
            return super().handle_request(issue)

class ExpertSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "expert":
            return "ExpertSupport: Handling expert issue"
        else:
            return super().handle_request(issue)


In [3]:
# __main__: Client
# Initiates the request and passes it 
# to the first handler in the chain.

# Create handlers
basic = BasicSupport()
advanced = AdvancedSupport()
expert = ExpertSupport()

# Set up Chain of Responsibility
# Basic -> Advanced -> Expert
basic.set_next_handler(advanced)
advanced.set_next_handler(expert)

# Client issues
# In this simple example, 
# each issue is a string that specifies the type of issue,
# i.e., we use it to decide which handler should process it
issues = ["basic", "advanced", "expert", "unknown"]

for issue in issues:
    print(f"Issue: {issue} - Response: {basic.handle_request(issue)}")
# Issue: basic - Response: BasicSupport: Handling basic issue
# Issue: advanced - Response: AdvancedSupport: Handling advanced issue
# Issue: expert - Response: ExpertSupport: Handling expert issue
# Issue: unknown - Response: No handler available for this issue


Issue: basic - Response: BasicSupport: Handling basic issue
Issue: advanced - Response: AdvancedSupport: Handling advanced issue
Issue: expert - Response: ExpertSupport: Handling expert issue
Issue: unknown - Response: No handler available for this issue


### Example: Card Chain

Example with card computer game. This example is similar but different to the previous one:

- In the Support example we took and issue and passed it to the Chain of Responsibility until the correct concrete handler is found.
- In this Card Game example, we create a chain of Creature modifiers, i.e., abilities, contained in a root modifier. Then, when we want to use the root, all modifiers are used and each calls to the next in the chain, except when a specific modifier with that function prevents it from propagating.

In [4]:
# Our Card-Creature
class Creature:
    def __init__(self, name, attack, defense):
        self.defense = defense
        self.attack = attack
        self.name = name

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

# A Creature-Card can be modified: 
# it collects/looses skills or objects: sword, poison, etc.
# This is where we define our Chain of Responsibility,
# it is our Abstract Modifier.
# We will inherit this base class to Concrete Modifiers.
class CreatureModifier:
    def __init__(self, creature):
        self.creature = creature
        self.next_modifier = None

    def add_modifier(self, modifier):
        if self.next_modifier:
            self.next_modifier.add_modifier(modifier)
        else:
            self.next_modifier = modifier

    # Here is where we apply the Chain of Responsibility
    # This handle() is called from the derived class objects
    def handle(self):
        if self.next_modifier:
            self.next_modifier.handle()

# Concrete Modifier
# Here, we define the concrete handle() methods
class DoubleAttackModifier(CreatureModifier):
    def handle(self):
        print(f'Doubling {self.creature.name}''s attack')
        self.creature.attack *= 2
        super().handle()


class IncreaseDefenseModifier(CreatureModifier):
    def handle(self):
        if self.creature.attack <= 2:
            print(f'Increasing {self.creature.name}''s defense')
            self.creature.defense += 1
        super().handle()

    
class NoBonusesModifier(CreatureModifier):
    def handle(self):
        print('No bonuses for you!')
        # Note that no super().handle() is called
        # That means we don't keep applying the Chain of Responsibility

In [5]:
# __main__
goblin = Creature('Goblin', 1, 1)
print(goblin)

# Abstract Handler: the root
root = CreatureModifier(goblin)

# Chain of Responsibility
root.add_modifier(NoBonusesModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))

# Add another modifier - no effect
root.add_modifier(IncreaseDefenseModifier(goblin))

root.handle()  # apply modifiers
print(goblin)

Goblin (1/1)
No bonuses for you!
Goblin (1/1)


### Example: Broker and Queries

In [6]:
from abc import ABC
from enum import Enum

# List of functions we can call
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2


class Query:
    def __init__(self, creature_name, what_to_query, default_value):
        self.value = default_value  # bidirectional
        self.what_to_query = what_to_query
        self.creature_name = creature_name

# Broker
class Game:
    def __init__(self):
        self.queries = Event()

    def perform_query(self, sender, query):
        self.queries(sender, query)


class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

    @property
    def attack(self):
        q = Query(self.name, WhatToQuery.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, WhatToQuery.DEFENSE, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'


class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle)

    def handle(self, sender, query):
        pass

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.game.queries.remove(self.handle)


class DoubleAttackModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2


class IncreaseDefenseModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.DEFENSE):
            query.value += 3


In [7]:
# __main__
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

with DoubleAttackModifier(game, goblin):
    print(goblin)
    with IncreaseDefenseModifier(game, goblin):
        print(goblin)

print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)
Strong Goblin (4/5)
Strong Goblin (2/2)


### Command

A Command is an object which represents an instruction to perform a particular action; it contains all the information necessary for the action to be taken.

- Ordinary statements are perishable:
  - They cannot undo member assignment:
  - They cannot directly serialize a sequence of actions (calls).
- We want an object that represents an operation.
  - Example: `person` should change its `age` to value `22`; the operation not only happens, but it is recoded it happened, along with who ordered it.
  - Example: `car` should `start()`.
- There are many uses for Commands: GUI commands, multi-level undo/redo, macro recording, and more!

### Example: Bank Account with Undo

In [8]:
from abc import ABC, abstractmethod
from enum import Enum

# Bank account class
# This class works nicely
# but we'd like to add the undo option
# for the opreations described in it.
# We can do that with a ledger/record.
# We can do that with a Command, too.
class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            # We need to return if it succeeded to avoid illegal operations
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'


# Optional, not necessary, but good pratice
class Command(ABC):
    # We could also use __call__, but invoke is more explicit
    @abstractmethod
    def invoke(self):
        pass

    @abstractmethod
    def undo(self):
        pass


# This Command class is a wrapper for the BankAccount
# which adds the Command functionality.
# The action on the BankAccount is not performed instantaneously
# but when we invoke() it.
# Additionally, we can undo the action.
class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account
        self.success = None

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        # To avoid illegal operations:
        # if the operation wasn't successful, don't allow undo
        if not self.success:
            return
        # Strictly speaking this is not correct
        # (you don't undo a deposit by withdrawing)
        # but it works for this demo, so...
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

In [9]:
# __main__
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

cmd.undo()
print('$100 deposit undone:', ba)

illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 1000)
illegal_cmd.invoke()
print('After impossible withdrawal:', ba)
illegal_cmd.undo()
print('After undo:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100
Withdrew 100, balance = 0
$100 deposit undone: Balance = 0
After impossible withdrawal: Balance = 0
After undo: Balance = 0


### Example: Bank Account with Composite Command = Macro

This example extends the previous one by adding a Composite construct, which simultaneously represents one Command and a Collection of Commands. That allows transferring money between different bank accounts easily. A Composite Commands is also known as Macro.

In [10]:
import unittest
from abc import ABC, abstractmethod
from enum import Enum


class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'


class Command(ABC):
    def __init__(self):
        self.success = False

    def invoke(self):
        pass

    def undo(self):
        pass


class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        super().__init__()
        self.amount = amount
        self.action = action
        self.account = account

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        if not self.success:
            return
        # Strictly speaking this is not correct
        # (you don't undo a deposit by withdrawing)
        # but it works for this demo, so...
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

In [11]:
# Try using this before using MoneyTransferCommand!
# This is a Composite: both the Command
# and a collection of commands are represented
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)

    def invoke(self):
        for x in self:
            x.invoke()

    def undo(self):
        for x in reversed(self):
            x.undo()


# This class solves the problem from the previous one

class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, from_acct, to_acct, amount):
        super().__init__([
            BankAccountCommand(from_acct,
                               BankAccountCommand.Action.WITHDRAW,
                               amount),
            BankAccountCommand(to_acct,
                               BankAccountCommand.Action.DEPOSIT,
                               amount)])

    def invoke(self):
        ok = True
        for cmd in self:
            if ok:
                cmd.invoke()
                ok = cmd.success
            else:
                cmd.success = False
        self.success = ok


In [12]:
class TestSuite(unittest.TestCase):
    def test_composite_deposit(self):
        ba = BankAccount()
        deposit1 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 1000)
        deposit2 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 1000)
        composite = CompositeBankAccountCommand([deposit1, deposit2])
        composite.invoke()
        print(ba)
        composite.undo()
        print(ba)

    def test_transfer_fail(self):
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        # composite isn't so good because of failure
        amount = 1000  # try 1000: no transactions should happen
        wc = BankAccountCommand(ba1, BankAccountCommand.Action.WITHDRAW, amount)
        dc = BankAccountCommand(ba2, BankAccountCommand.Action.DEPOSIT, amount)

        transfer = CompositeBankAccountCommand([wc, dc])

        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)  # end up in incorrect state
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)

    def test_better_tranfer(self):
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        amount = 1000

        transfer = MoneyTransferCommand(ba1, ba2, amount)
        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)
        print(transfer.success)

In [13]:
# __main__
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...

ba1: Balance = 100 ba2: Balance = 0
ba1: Balance = 100 ba2: Balance = 0
False
Deposited 1000, balance = 1000
Deposited 1000, balance = 2000
Balance = 2000
Withdrew 1000, balance = 1000
Withdrew 1000, balance = 0
Balance = 0
Deposited 1000, balance = 1000
ba1: Balance = 100 ba2: Balance = 1000
Withdrew 1000, balance = 0
ba1: Balance = 100 ba2: Balance = 0



----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


## Interpreter

The Interpreter is related to the processing that text requires to interpret what to do with it:

- Textual input needs to be processed; e.g., code turned in to OOP structures (compilers).
- Some examples:
  - Programming language compilers, interpreters and IDEs.
  - HTML, XML, and similar
  - Numeric expressions (`3 + 4/5`)
  - Regular expressions

Often, the processing is done to obtain structured meaning. To that end:

- We separate it into lexical tokens (*lexing*)
- and then interpret the resulting sequences (*parsing*).

In the following an example numeric expressions with `+` and `-` operations are processed with an interpreter.

### Lexing

In this section, we separate the input string into a sequence of known tokens. That's called *lexing*.

In [14]:
from enum import Enum, auto


class Token:
    # Class-level enum
    class Type(Enum):
        INTEGER = auto()
        PLUS = auto()
        MINUS = auto()
        LPAREN = auto() # (
        RPAREN = auto() # )

    def __init__(self, type, text):
        self.type = type
        self.text = text

    def __str__(self):
        return f'`{self.text}`'


# Lexing
def lex(input):
    result = []

    i = 0
    while i < len(input):
        if input[i] == '+':
            result.append(Token(Token.Type.PLUS, '+'))
        elif input[i] == '-':
            result.append(Token(Token.Type.MINUS, '-'))
        elif input[i] == '(':
            result.append(Token(Token.Type.LPAREN, '('))
        elif input[i] == ')':
            result.append(Token(Token.Type.RPAREN, ')'))
        else:  # must be a number
            digits = [input[i]]
            for j in range(i + 1, len(input)):
                if input[j].isdigit():
                    digits.append(input[j])
                    i += 1
                else:
                    result.append(Token(Token.Type.INTEGER,
                                        ''.join(digits)))
                    break
        i += 1

    return result

### Parsing

In this section, we take the sequence of tokens and interpret their meaning, aka. *parsing*.

In [15]:
class Integer:
    def __init__(self, value):
        self.value = value


class BinaryOperation:
    class Type(Enum):
        ADDITION = 0
        SUBTRACTION = 1

    def __init__(self):
        self.type = None
        self.left = None
        self.right = None

    @property
    def value(self):
        if self.type == self.Type.ADDITION:
            return self.left.value + self.right.value
        elif self.type == self.Type.SUBTRACTION:
            return self.left.value - self.right.value


def parse(tokens):
    result = BinaryOperation()
    have_lhs = False # left-hand-side
    i = 0
    while i < len(tokens):
        token = tokens[i]

        if token.type == Token.Type.INTEGER:
            integer = Integer(int(token.text))
            if not have_lhs:
                result.left = integer
                have_lhs = True
            else:
                result.right = integer
        elif token.type == Token.Type.PLUS:
            result.type = BinaryOperation.Type.ADDITION
        elif token.type == Token.Type.MINUS:
            result.type = BinaryOperation.Type.SUBTRACTION
        elif token.type == Token.Type.LPAREN:  # note: no if for RPAREN
            j = i
            while j < len(tokens):
                if tokens[j].type == Token.Type.RPAREN:
                    break
                j += 1
            # preprocess subexpression
            subexpression = tokens[i + 1:j]
            element = parse(subexpression)
            if not have_lhs:
                result.left = element
                have_lhs = True
            else:
                result.right = element
            i = j  # advance
        i += 1
    return result

In [16]:
def eval(input):
    tokens = lex(input)
    print(' '.join(map(str, tokens)))

    parsed = parse(tokens)
    print(f'{input} = {parsed.value}')

# __main__
eval('(13+4)-(12+1)')
eval('1+(3-4)')

# this won't work
eval('1+2+(3-4)')

`(` `13` `+` `4` `)` `-` `(` `12` `+` `1` `)`
(13+4)-(12+1) = 4
`1` `+` `(` `3` `-` `4` `)`
1+(3-4) = 0
`1` `+` `2` `+` `(` `3` `-` `4` `)`
1+2+(3-4) = 0


## Iterator

- Iteration (traversal) is a core functionality of various data structures; note that some data structures (like trees) don't have a straightforward iteration process.
- An iterator is a class that facilitates the traversal
  - Keeps a reference to the current element.
  - Knows how to move to a different element.
- The iterator protocol requires:
  - `__iter__()` to expose the iterator, which uses
  - `__next__()` to return each of the iterated elements or
  - `raise StopIteration` when it's done.
- In Python, we have the keyword `token` which makes iteration very easy.

In the following, the Iterator of a Binary Tree is shown.

In [17]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.right = right
        self.left = left
        self.value = value

        self.parent = None

        if left:
            self.left.parent = self
        if right:
            self.right.parent = self

    # We expose the tree/Node with __iter__
    def __iter__(self):
        # We use the Iterator we want here
        return InOrderIterator(self)

# Recall we can perform 3 traversals in a tree
#       1
#     /   \
#    2     3
# in-order: 2-1-3 (we implement only this one)
# pre-order: 1-2-3
# post-order: 2-3-1
class InOrderIterator:
    def __init__(self, root):
        self.root = self.current = root
        self.yielded_start = False
        while self.current.left:
            self.current = self.current.left

    # This is a tricky implementation of in-order
    def __next__(self):
        if not self.yielded_start:
            self.yielded_start = True
            return self.current

        if self.current.right:
            self.current = self.current.right
            while self.current.left:
                self.current = self.current.left
            return self.current
        else:
            p = self.current.parent
            while p and self.current == p.right:
                self.current = p
                p = p.parent
            self.current = p
            if self.current:
                return self.current
            else:
              raise StopIteration

def traverse_in_order(root):
    # This is the usual implementation of in-order
    # It leverages the use of yield
    def traverse(current):
        if current.left:
            for left in traverse(current.left):
                yield left
        yield current
        if current.right:
            for right in traverse(current.right):
                yield right
    for node in traverse(root):
        yield node

In [18]:
# __main__

# Recall we can perform 3 traversals in a tree
#       1
#     /   \
#    2     3
# in-order: 2-1-3 (we implement only this one)
# pre-order: 1-2-3
# post-order: 2-3-1
root = Node(1,
            Node(2),
            Node(3))

# We can iterate with iter and next
it = iter(root)
print([next(it).value for x in range(3)])

# Or, since we have __iter__, in a loop
for x in root:
    print(x.value)

for y in traverse_in_order(root):
    print(y.value)

[2, 1, 3]
2
1
3
2
1
3


## Mediator

The Mediator facilitates communication between different components without them necessarily being aware of each other or having direct (reference) access to each other.

- Components may go in and out of a system at any time. For instance:
  - Chatroom participants
  - Players in an online game
- It makes no sense for rthem to have direct reference to one another
  - Because those references may go dead
- Solution: we can have them all refer to some central component that facilitates communication, i.e., the Mediator.
  - All components in the system refer to the Mediator.
  - The Mediator engages in bidirectional communication with its connected components.
  - The Mediator has functions that the components can call.
  - The components have functions that the Mediator can call.
  - We can also work with events.

### Example: Chatroom

In [19]:
class Person:
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None

    def receive(self, sender, message):
        s = f'{sender}: {message}'
        print(f'[{self.name}\'s chat session] {s}')
        self.chat_log.append(s)

    def say(self, message):
        self.room.broadcast(self.name, message)

    def private_message(self, who, message):
        self.room.message(self.name, who, message)

# This is our Mediator,
# since it allows private and broadcasting messages
class ChatRoom:
    def __init__(self):
        self.people = []

    # Broadcast = message to all
    # We should send a message to everybody
    # except the source
    # We traverse all users and make them receive()
    # the message
    def broadcast(self, source, message):
        for p in self.people:
            if p.name != source:
                p.receive(source, message)

    def join(self, person):
        join_msg = f'{person.name} joins the chat'
        self.broadcast('room', join_msg)
        person.room = self
        self.people.append(person)

    # Message = message to one specififc person
    # We traverse all users and make _only_
    # the specific user receive()
    # the message
    def message(self, source, destination, message):
        for p in self.people:
            if p.name == destination:
                p.receive(source, message)

In [20]:
# __main__
room = ChatRoom()

john = Person('John')
jane = Person('Jane')

room.join(john)
room.join(jane)

john.say('hi room')
jane.say('oh, hey john')

simon = Person('Simon')
room.join(simon)
simon.say('hi everyone!')

jane.private_message('Simon', 'glad you could join us!')

[John's chat session] room: Jane joins the chat
[Jane's chat session] John: hi room
[John's chat session] Jane: oh, hey john
[John's chat session] room: Simon joins the chat
[Jane's chat session] room: Simon joins the chat
[John's chat session] Simon: hi everyone!
[Jane's chat session] Simon: hi everyone!
[Simon's chat session] Jane: glad you could join us!


### Example: Game with Events

In [21]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

# This is our Mediator:
# centrally available component that makes
# possible the communication between entities
class Game:
    def __init__(self):
        self.events = Event()

    def fire(self, args):
        self.events(args)


class GoalScoredInfo:
    def __init__(self, who_scored, goals_scored):
        self.goals_scored = goals_scored
        self.who_scored = who_scored


class Player:
    def __init__(self, name, game):
        self.name = name
        self.game = game
        self.goals_scored = 0

    def score(self):
        self.goals_scored += 1
        args = GoalScoredInfo(self.name, self.goals_scored)
        self.game.fire(args)


class Coach:
    def __init__(self, game):
        game.events.append(self.celebrate_goal)

    def celebrate_goal(self, args):
        if isinstance(args, GoalScoredInfo) and args.goals_scored < 3:
            print(f'Coach says: well done, {args.who_scored}!')


In [22]:
# __main__
game = Game()
player = Player('Sam', game)
coach = Coach(game)

player.score()  # Coach says: well done, Sam!
player.score()  # Coach says: well done, Sam!
player.score()  # ignored by coach

Coach says: well done, Sam!
Coach says: well done, Sam!


## Memento

- An object or a system can go through several changes.
  - Example: a bank account gets deposits and withdrawals.
- There are different ways of navigating those changes.
  - One way is to record every change (Command) and teach a command to *undo* itself.
  - Another is to simply save snapshots of the system = Memento.
- The Memento pattern is a token/handle class representing the system state.
  - It lets us roll back to the state when the token was generated.
  - It may or may not expose state information.

So, basically, the Memento pattern is like a history, i.e., a list of all states along the time. Whenever we perform a change, a state is saved so that we can go back/forth in the history of object states.

### Example: Bank Account

In this example, for every deposit we do in a bank account, a Memento (state = balance) object is returned. Given a Memento of a bank account, we can restore that bank account with the state of the Memento.

In [23]:
# We could call it BanckAccountSnapshot
# In this simple example, the state is just the balance
class Memento:
    def __init__(self, balance):
        self.balance = balance


class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return Memento(self.balance)

    def restore(self, memento):
        self.balance = memento.balance

    def __str__(self):
        return f'Balance = {self.balance}'

In [24]:
# __main__
ba = BankAccount(100)
m1 = ba.deposit(50)
m2 = ba.deposit(25)
print(ba)

# restore to m1
ba.restore(m1)
print(ba)

# restore to m2
ba.restore(m2)
print(ba)
# Balance = 175
# Balance = 150
# Balance = 175

Balance = 175
Balance = 150
Balance = 175


### Example: Bank Account with Undo and Redo

This example builds on the previous one. We build a list of all the changes (a list of Mementos) in the bank account.

In [25]:
class Memento:
    def __init__(self, balance):
        self.balance = balance


class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.changes = [Memento(self.balance)] # our history
        self.current = 0 # index where we are in changes

    def deposit(self, amount):
        self.balance += amount
        m = Memento(self.balance)
        self.changes.append(m)
        self.current += 1
        return m

    def restore(self, memento):
        if memento:
            self.balance = memento.balance
            self.changes.append(memento)
            self.current = len(self.changes)-1

    def undo(self):
        # We go back one step in history/changes
        if self.current > 0:
            self.current -= 1
            m = self.changes[self.current]
            self.balance = m.balance
            return m
        return None

    def redo(self):
        # We go one step forward in history/changes
        if self.current + 1 < len(self.changes):
            self.current += 1
            m = self.changes[self.current]
            self.balance = m.balance
            return m
        return None

    def __str__(self):
        return f'Balance = {self.balance}'

In [26]:
# __main__
ba = BankAccount(100)
ba.deposit(50)
ba.deposit(25)
print(ba)

ba.undo()
print(f'Undo 1: {ba}')
ba.undo()
print(f'Undo 2: {ba}')
ba.redo()
print(f'Redo 1: {ba}')
# Balance = 175
# Undo 1: Balance = 150
# Undo 2: Balance = 100
# Redo 1: Balance = 150

Balance = 175
Undo 1: Balance = 150
Undo 2: Balance = 100
Redo 1: Balance = 150


## Observer

The Observer is probably the most common pattern.

- We need to be informed when certain things happen
  - Object's property changes
  - Object does something
  - Some external event occurs
- We want to listen to those events and we want to be notified when they occur
  - Notifications should include useful data
- We want to unsubscribe from events we're no longer interested in

The Observer is an object that wishes to be informed about events happening in the system. The entity generating the events is an *observable*.

One common use-case is when a class property changes; we can use Python `@property.setter` decorator to invoke callable `Event` objects, which consist of lists of functions which are run when an event occurs.

Also, note that:

- In general, the Observer is an intrusive approach, i.e., the observable must provide an event to subscribe to.
- Subscription and unsubscription is handled with addition/removal of items in a list.
- Property change notifications are easy, but when properties depend on others, notifications start to be a bit more tricky: the dependent properties must be handled in the `@property.setter` of the main/original attribute that affects all ohers.

### Events

In this approach to implement an Observer we define an `Event` class derived from `list` which collects callable functions, which are kind of subscribers. In the example, a `Person` is defined with and `Event` type/class for the occasion when he/she falls ill. When that occurs, all the callable subscribers (i.e., functions) from the `Event` are called.

In [27]:
# We have a Person class
# We want an Event to occur, we one Person falls ill
class Person:
    def __init__(self, name, address):
        self.name = name
        self.address = address
        self.falls_ill = Event()

    def catch_a_cold(self):
        self.falls_ill(self.name, self.address)


# In the Event class, we have subscribers
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


def call_doctor(name, address):
    print(f'A doctor has been called to {address}')


In [28]:
# __main__
person = Person('Sherlock', '221B Baker St')

# Here, we're accessing Event,
# which is a list of items!
# Each item is a function
# which can be called!
# Here, the callable items just print something
# but similarly, they could perform other tasks,
# e.g., send an email!
person.falls_ill.append(lambda name, addr: print(f'{name} is ill'))
person.falls_ill.append(call_doctor)

# We call the functions by calling the Event
# which is a list of functions = subscribers
person.catch_a_cold()

# And we can remove subscriptions too
person.falls_ill.remove(call_doctor)
# So only one callable item = subscriber = function is run
person.catch_a_cold()

Sherlock is ill
A doctor has been called to 221B Baker St
Sherlock is ill


### Property Observers

This example shows how the changes in class attributes can be notified by using the `@property` decorator in Python. Basically, the same `Event` class as before is used via the `@property.setter`.

In [36]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


# Base class which contains an Event class instance
# Person is derived from it!
class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()


# Person is derived from the Observer
# so that we have the Events baked in
class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age

    @property
    def age(self):
        return self._age

    # In the property setter
    # we trigger the Event
    # which is related to changes in
    # the age attribute
    @age.setter
    def age(self, value):
        if self._age == value:
            return
        self._age = value
        self.property_changed('age', value)


# This class uses the Observer
# to check when people can drive
class TrafficAuthority:
    def __init__(self, person):
        self.person = person
        person.property_changed.append(self.person_changed)

    def person_changed(self, name, value):
        if name == 'age':
            if value < 16:
                print('Sorry, you still cannot drive')
            else:
                print('Okay, you can drive now')
                self.person.property_changed.remove(self.person_changed)

In [37]:
# __main__
p = Person()
ta = TrafficAuthority(p)
for age in range(14, 20):
    print(f'Setting age to {age}')
    p.age = age

Setting age to 14
Sorry, you still cannot drive
Setting age to 15
Sorry, you still cannot drive
Setting age to 16
Okay, you can drive now
Setting age to 17
Setting age to 18
Setting age to 19


### Property Dependencies

Property Observers can face difficulties when properties are dependent among them. This example shows how to solve those cases.

In [33]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()


class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age

    # We add a boolean property to denote whether a Person can vote
    # which clearly depends on the age
    @property
    def can_vote(self):
        return self._age >= 18

    @property
    def age(self):
        return self._age

    # The original property from which the dependent ones
    # hand needs to be modified:
    # We track when the dependent properties change
    # and notify the observer via the Event function calls
    @age.setter
    def age(self, value):
        if self._age == value:
            return

        # We cash the old can_vote
        old_can_vote = self.can_vote

        self._age = value
        self.property_changed('age', value)

        # If the new can_vote is different than the old
        # then, we notify the observer
        if old_can_vote != self.can_vote:
            self.property_changed('can_vote', self.can_vote)

In [34]:
# __main__
def person_changed(name, value):
    if name == 'can_vote':
        print(f'Voting status changed to {value}')

p = Person()
p.property_changed.append(
    person_changed
)

for age in range(16, 21):
    print(f'Changing age to {age}')
    p.age = age


Changing age to 16
Changing age to 17
Changing age to 18
Voting status changed to True
Changing age to 19
Changing age to 20


## State (Machine)

Consider an ordinary telephone:

- What you do with it depends on the state of the phone/line
  - If it's ringing or you want to make a call, you can pick it up.
  - The phone must be off the hook to talk/make a call.
  - If you try calling someone and it's busy, you put the handset down.
- Changes in state can be explicit or in response to an event (Observer pattern).

Following the telephone's analogy, the State is a pattern in which the object's behavior is determined by its state. An object transitions from one state to another (something needs to trigger that transition). A formalized construct which manages state and transitions is called a *(finite) state machine*.

The best way of implementing State Machines is to define `State` and `Trigger` classes/enums:

- We define all possible `States`.
- We define all possible events or `Triggers` that can occur.
- We map to each `State` the `Trigger` events that can apply to them and to which `State` they transition when they occur.
- Everything can be packed into a `while` loop where each `State`-`Trigger` pair is checked, and we can sophisticate the logic as we please:
  - Define entry/exit `States`.
  - Gaurd conditions for enabling/disabling transitions.
  - Default action when no transitions are found for an event/`Trigger`.

### Classic State

This example is a bit unrealistic/unpractical, i.e., we would not implement it in real life, because it is a too complicated solution for a simple case in which the `State` of an `ON/OFF` switch is modeled. Of course, we can find a more complex use-case which follows this `State -> XState` structure.

This type of State Machine implementations were done some decades ago, but nowadays not anymore. However, it's maybe good to keep in mind how people used to solve this things beforehand.

In [40]:
from abc import ABC


# Light switch
# The State classes are used internally
# and we expose all the transition buttons
# to the Switch here
class Switch:
    def __init__(self):
        self.state = OffState()

    # We force a transition of State
    def on(self):
        self.state.on(self)

    # Trasition of State
    def off(self):
        self.state.off(self)


# Abstract
class State(ABC):
    def on(self, switch):
        print('Light is already on')

    def off(self, switch):
        print('Light is already off')


# We derive each State
# Note that we define only State transtion methods!
# OnState: off(): on -> off
# If a transition to self is done, the parent method
# is called! E.g., OnState.on = State.on
class OnState(State):
    def __init__(self):
        print('Light turned on')

    # Note we take a Switch as argument
    def off(self, switch):
        print('Turning light off...')
        switch.state = OffState()


# We derive each State
# Note that we define only State transtion methods!
# OffState: on(): off -> on
# If a transition to self is done, the parent method
# is called! E.g., OffState.on = State.off
class OffState(State):
    def __init__(self):
        print('Light turned off')

    # Note we take a Switch as argument
    def on(self, switch):
        print('Turning light on...')
        switch.state = OnState()

In [39]:
# __main__
sw = Switch()

sw.on()  # Turning light on...
         # Light turned on

sw.off()  # Turning light off...
          # Light turned off

sw.off()  # Light is already off

Light turned off
Turning light on...
Light turned on
Turning light off...
Light turned off
Light is already off


### Handmade State Machine

In this example a phone call is modeled with a State Machine. `State` and `Trigger` `Enums` are defined and a `rules` dictionary which maps each current `State` to all possible `(Trigger, NewState)` pairs. Then, we check in a loop the `Trigger` for the current `State` and perform a State Transition associated with each `Trigger`.

In [9]:
from enum import Enum, auto


class State(Enum):
    OFF_HOOK = auto() # handset lifted from the cradle = descolgado
    CONNECTING = auto()
    CONNECTED = auto()
    ON_HOLD = auto()
    ON_HOOK = auto() # handset placed on the cradle = colgado


# Triggers are the things that cause the State transitions
class Trigger(Enum):
    CALL_DIALED = auto()
    HUNG_UP = auto()
    CALL_CONNECTED = auto()
    PLACED_ON_HOLD = auto()
    TAKEN_OFF_HOLD = auto()
    LEFT_MESSAGE = auto()

In [11]:
# __main__
# Now for each State, we need to define which
# Triggers would cause to transition to a new State
# We have a list of pairs for each State:
# (Trigger event, State we transition to)
# Everything is packed in a dictionary Dict[State, List[Trigger, State]]
rules = {
    State.OFF_HOOK: [
        (Trigger.CALL_DIALED, State.CONNECTING)
    ],
    State.CONNECTING: [
        (Trigger.HUNG_UP, State.ON_HOOK),
        (Trigger.CALL_CONNECTED, State.CONNECTED)
    ],
    State.CONNECTED: [
        (Trigger.LEFT_MESSAGE, State.ON_HOOK),
        (Trigger.HUNG_UP, State.ON_HOOK),
        (Trigger.PLACED_ON_HOLD, State.ON_HOLD)
    ],
    State.ON_HOLD: [
        (Trigger.TAKEN_OFF_HOLD, State.CONNECTED),
        (Trigger.HUNG_UP, State.ON_HOOK)
    ]
}

# Starting State: When the State Machine is initiated
state = State.OFF_HOOK
# Ending State; in some State Machines
# exit_state does not occur, e.g., trading bots.
# Here, as soon as a person sets the phone on hook
# we are done!
exit_state = State.ON_HOOK

while state != exit_state:
    print(f'The phone is currently {state}')

    # Check all rules to find current state
    for i in range(len(rules[state])):
        t = rules[state][i][0]
        print(f'{i}: {t}')

    # Request Trigger input
    idx = int(input('Select a trigger:'))
    # Perform State Transiton with Trigger
    s = rules[state][idx][1]
    state = s

print('We are done using the phone.')
# We enter idx in List[] of rules[State], e.g. 0
# The phone is currently State.OFF_HOOK
# 0: Trigger.CALL_DIALED
# The phone is currently State.CONNECTING
# 0: Trigger.HUNG_UP
# 1: Trigger.CALL_CONNECTED
# We are done using the phone.

The phone is currently State.OFF_HOOK
0: Trigger.CALL_DIALED
The phone is currently State.CONNECTING
0: Trigger.HUNG_UP
1: Trigger.CALL_CONNECTED
We are done using the phone.


### Switch-Based State Machine

In this example, the State Machine of a lock is implemented following the `switch`-based approach. Python doesn't have a `switch` statement, but we can imlement it using `if-else` or `dicts`.

The advantage of this approach is that we just need to define only `State`, and all the transitions are handled in a `while`-loop. However, it works only for simple State Machines; in complex cases, it is better to define `Triggers` and transition `rules`, as done beforehand.

In [13]:
from enum import Enum, auto


class State(Enum):
    LOCKED = auto()
    FAILED = auto()
    UNLOCKED = auto()


In [15]:
# __main__
code = '1234'
state = State.LOCKED
entry = ''

while True:
    if state == State.LOCKED:
        entry += input(entry)

        if entry == code:
            state = State.UNLOCKED

        if not code.startswith(entry):
            # the code is wrong
            state = State.FAILED
    elif state == State.FAILED:
        print('\nFAILED')
        entry = ''
        state = State.LOCKED
    elif state == State.UNLOCKED:
        print('\nUNLOCKED')
        break


FAILED

UNLOCKED


## Strategy

Many algorithms can be separated into two layers: high-level and low-level; while high-level deals with the overall goal, the low-level deals with the details. The Strategy makes use of that sepration and enables to select the exact behavior of a system at run-time.

Example: the algorithm of making tea can be decomposed into:

- the process of making a hot beverage (boil water, pout into a cut)
- and tea-specific steps (put teabag into water, etc.)

The high-level algorithm is the one which is reused and provided to the user (prepare hot beverage), who then adds/selects low-level parts (coffe, tea, hot chocolate). Each of the low-level options is called a Strategy, which supports specifics associated with the low-level option:

- Tea-Strategy
- Coffee-Strategy
- Hot-chocolate-Strategy
- etc.

So, in summary, when we use Strategies:

- We define an algorithm at a high level.
- We define the interface we expect each Strategy to follow.
- We provide for dynamic composition of Strategies in the resulting object.

### Example: Text Processor

This example is a simple text processor which formats a list of bullet points to be in Markdown or HTML. The Strategy pattern is used to specify the low-level text foramtting of the list. That way, we define the high-level code and inject low-level Strategy objects, which are implemented somewhere else. In this case, the Strategy = Format (HTML/Markdown).

In [18]:
from abc import ABC
from enum import Enum, auto


class OutputFormat(Enum):
    MARKDOWN = auto()
    HTML = auto()


# Abstract base class for all Strategies = Formats
# Not required but a good idea
class ListStrategy(ABC):
    def start(self, buffer): pass
    def end(self, buffer): pass
    def add_list_item(self, buffer, item): pass


# Markdown lists are very easy
class MarkdownListStrategy(ListStrategy):

    def add_list_item(self, buffer, item):
        buffer.append(f' * {item}\n')


# HTML lists are a bit more complicated
# bacause they have start/end tags 
# for both the complete list
# and each item
class HtmlListStrategy(ListStrategy):

    def start(self, buffer):
        buffer.append('<ul>\n')

    def end(self, buffer):
        buffer.append('</ul>\n')

    def add_list_item(self, buffer, item):
        buffer.append(f'  <li>{item}</li>\n')


# This text processor is a high-level API
# in which we can choose the low-level Strategy
# i.e., the format, in this case
class TextProcessor:
    def __init__(self, list_strategy=HtmlListStrategy()):
        self.buffer = [] # all the text goes here
        self.list_strategy = list_strategy

    # We add the items of a list, which are appended to the buffer
    def append_list(self, items):
        self.list_strategy.start(self.buffer)
        for item in items:
            self.list_strategy.add_list_item(
                self.buffer, item
            )
        self.list_strategy.end(self.buffer)

    # We set the format at the beginning
    # which instantiates the desired Strategy = Format
    def set_output_format(self, format):
        if format == OutputFormat.MARKDOWN:
            self.list_strategy = MarkdownListStrategy()
        elif format == OutputFormat.HTML:
            self.list_strategy = HtmlListStrategy()

    def clear(self):
        self.buffer.clear()

    def __str__(self):
        return ''.join(self.buffer)

In [20]:
# __main__
items = ['foo', 'bar', 'baz']

tp = TextProcessor()
tp.set_output_format(OutputFormat.MARKDOWN)
tp.append_list(items)
print(tp)
#  * foo
#  * bar
#  * baz

tp.set_output_format(OutputFormat.HTML)
tp.clear()
tp.append_list(items)
print(tp)
# <ul>
#   <li>foo</li>
#   <li>bar</li>
#   <li>baz</li>
# </ul>

 * foo
 * bar
 * baz

<ul>
  <li>foo</li>
  <li>bar</li>
  <li>baz</li>
</ul>



## Template Method

A Template is a high-level blueprint for an algorithm to be completed by inheritors. In other words, it is equivalent to the Strategy method, but it's implemented via inheritance: it allows us to define the skeleton of the algorithm with concrete implementations defined in subclasses.

- Algorithms can be decomposed into common parts (high-level) + specifics (low-level).
- The Strategy pattern accomplishes the separation of high/low-level parts throuhg composition:
  - High-level algorithm expects strateges to conform to an interface.
  - Concrete implementations implement this interface and are used.
- The Template Method pattern does the same thing as Strategy, but using inheritance
  - The overall algorithm is defined in the base class; it uses abstract members.
  - Inheritors override the abstract members.
  - Template method invoked to get the work done.

The Template Method is usually defined in the base class and not overriden in derived concrete classes; instead, the abstract methods used by it are overriden. That way, the high-level logic is already defined in the Template Method of the base class.

### Example: Chess Game

In [24]:
from abc import ABC


# Our base class = our Skeleton
# It defines the structure of any/many game/s
class Game(ABC):

    def __init__(self, number_of_players):
        self.number_of_players = number_of_players
        self.current_player = 0

    # This is the Template Method!
    def run(self):
        self.start()
        while not self.have_winner:
            self.take_turn()
        print(f'Player {self.winning_player} wins!')

    def start(self): pass

    @property
    def have_winner(self): pass

    def take_turn(self): pass

    @property
    def winning_player(self): pass


# This is our inherited class.
# We won't override our Template Method (run)
# but instead, all the other methods used 
# by it: have_winner, take_turn, winning_player
class Chess(Game):
    def __init__(self):
        super().__init__(2)
        self.max_turns = 10
        self.turn = 1

    def start(self):
        print(f'Starting a game of chess with {self.number_of_players} players.')

    @property
    def have_winner(self):
        return self.turn == self.max_turns

    def take_turn(self):
        print(f'Turn {self.turn} taken by player {self.current_player}')
        self.turn += 1
        self.current_player = 1 - self.current_player

    @property
    def winning_player(self):
        return self.current_player

In [23]:
# __main__
chess = Chess()
chess.run()
# Starting a game of chess with 2 players.
# Turn 1 taken by player 0
# Turn 2 taken by player 1
# Turn 3 taken by player 0
# Turn 4 taken by player 1
# Turn 5 taken by player 0
# Turn 6 taken by player 1
# Turn 7 taken by player 0
# Turn 8 taken by player 1
# Turn 9 taken by player 0
# Player 1 wins!

Starting a game of chess with 2 players.
Turn 1 taken by player 0
Turn 2 taken by player 1
Turn 3 taken by player 0
Turn 4 taken by player 1
Turn 5 taken by player 0
Turn 6 taken by player 1
Turn 7 taken by player 0
Turn 8 taken by player 1
Turn 9 taken by player 0
Player 1 wins!


## Visitor

Sometimes we need to define a new operation/bahavior on an entire class hierarchy (i.e., in several classes with a hierarchical dependency): that's exactly what the Visitor pattern makes possible.

- Example: we want to make a document model printable to HTML/Markdown.
- We don't want to keep modifying every class in the hierarchy.
- We need access to the non-common aspects of classes in the hierarchy.
- We create an external component to handle rendering.
  - But we want to avoid explicit type checks.

The Visitor is a components that knows how to traverse a data structure composed of (possibly related) types.

### Intrusive (Bad) Visitor

In this example we capture expressions like `1 + (2+3)` in Object-Oriented fashion:

- We have left/right terms.
- Each term can be an expression, too.
- We want to print the complete expression.
- We want to evaluate the complete expression.

This Visitor example is *intrusive* because be break the Open-Closed Principle: We modify the classes to add additional functionality, in this case they print themselves.

The modification is the addition of the `print()` method, which outputs to a `buffer: list` all the components of the expression. The problem is that

- We need to add a `print()` method to all classes in the hierarchy so that the expression can be printed.
- Adding a `print()` method post-hoc breaks the Open-Closed principle and it can be cumbersome: if the class hierarchy is large, we have much work to do.
- Something similar happens with `eval()`.

In [27]:
class DoubleExpression:
    def __init__(self, value):
        self.value = value

    # This print is intrusive (breaks Open-Close principle)
    # because we add it afterwards
    # and we need it (with custom implementations)
    # in all classes related to this
    def print(self, buffer):
        buffer.append(str(self.value))

    # This is also an intrusive solution
    def eval(self): return self.value


class AdditionExpression:
    def __init__(self, left, right):
        self.right = right
        self.left = left

    # This print is intrusive (breaks Open-Close principle)
    # because we add it afterwards
    # and we need it (with custom implementations)
    # in all classes related to this
    def print(self, buffer):
        buffer.append('(')
        self.left.print(buffer)
        buffer.append('+')
        self.right.print(buffer)
        buffer.append(')')

    # This is also an intrusive solution
    def eval(self):
        return self.left.eval() + self.right.eval()

In [29]:
# __main__
# This represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
buffer = []
# We print the expression:
# Intrusive print() method
e.print(buffer)
print(''.join(buffer), '=', e.eval())

# This breaks OCP: requires we modify the entire hierarchy
# What is more likely: new type or new operation?

(1+(2+3)) = 6


### Reflective Visitor

An improvement to the previous approach is presented below. Here, a new class `ExpressionPrinter` is created which takes care of the printing: it contains the functionality in a separate class. The solution still breaks the Open-Closed principle, because we need to create `M (print, eval) x N (DoubleExpression, AdditionExpession)` new classes which are modified every time we extend the functionalities, but it's still an enhancement.

This example, in addition, *injects* a `print()` function from `ExpressionPrinter` to an abstract base class `Expression`, which is used by the derived `DoubleExpression` and `AdditionExpression`.

The example is called *reflective* because types need to be checked. From the [Wikipedia](https://en.wikipedia.org/wiki/Reflective_programming):

> In computer science, reflective programming or reflection is the ability of a process to examine, introspect, and modify its own structure and behavior.

Another example of *reflection*, from the [Wikipedia](https://en.wikipedia.org/wiki/Reflective_programming)

```python
class Foo:
    def __init__(self):
        pass
    def hello(self):
        print("Hello!")

# Without reflection
obj = Foo()
obj.hello()

# With reflection
obj = globals()["Foo"]()
getattr(obj, "hello")()

# With eval
eval("Foo().hello()")
```

Reflective Visitor Example:

In [35]:
from abc import ABC


# We use this abstract base class to join
# all Expression* classes in a hierarchy
# Later (in ExpressionPrinter) we will
# define a print() method shared among all
# derived classes!
class Expression(ABC):
    pass


# Note we derive the class from Expression
class DoubleExpression(Expression):
    def __init__(self, value):
        self.value = value


# Note we derive the class from Expression
class AdditionExpression(Expression):
    def __init__(self, left, right):
        self.right = right
        self.left = left


# This still breaks OCP 
# because new types require M×N modifications.
# If we add new types
# we need to modify ExpressionPrinter.
# If we add new operations (print, eval)
# we need to reate anew class.
class ExpressionPrinter:
    # Static method = class-level method, 
    # we don't need to instantiate a class object
    @staticmethod
    def print(e, buffer):
        """ Will fail silently on a missing case. """
        if isinstance(e, DoubleExpression):
            buffer.append(str(e.value))
        elif isinstance(e, AdditionExpression):
            buffer.append('(')
            ExpressionPrinter.print(e.left, buffer)
            buffer.append('+')
            ExpressionPrinter.print(e.right, buffer)
            buffer.append(')')

    # We (re-)define Expression.print in ExpressionPrinter
    # so that we can call print() from DoubleExpression and AdditionExpression
    # because the are inherited from Expression!
    Expression.print = lambda self, b: ExpressionPrinter.print(self, b)

In [38]:
# __main__
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)

buffer = []
# print() is a static method = class-level method, 
# we don't need to instantiate a class object
ExpressionPrinter.print(e, buffer)
print("ExpressionPrinter.print:", ''.join(buffer))

# Since we inherited all Expression* from the ABC Expression
# and we (re-)define Expression.print in ExpressionPrinter
# we can still call print() from DoubleExpression and AdditionExpression
# because the are inherited from Expression!
buffer = []
e.print(buffer)
print("e.print:", ''.join(buffer))


ExpressionPrinter.print: (1+(2+3))
e.print: (1+(2+3))


### Classic Visitor

This implementation follows the example from the Gang of Four, which uses a [*double dispatch*](https://en.wikipedia.org/wiki/Double_dispatch), a technical term to describe the process of choosing the method to invoke based both on receiver and argument types.

As in the previous examples, we use an explicit new class for printing: `ExpressionPrinter`. However, a decorator `@visitor` is previously defined which is used to overload the `visit()` method in `ExpressionPrinter`.

In [50]:
# VISITOR CODE
# Example taken from 
# https://tavianator.com/the-visitor-pattern-in-python/

def _qualname(obj):
    """Get the fully-qualified name of an object (including module)."""
    return obj.__module__ + '.' + obj.__qualname__


def _declaring_class(obj):
    """Get the name of the class that declared an object."""
    name = _qualname(obj)
    return name[:name.rfind('.')]


# Stores the actual visitor methods
_methods = {}


# Delegating visitor implementation
def _visitor_impl(self, arg):
    """Actual visitor method implementation."""
    method = _methods[(_qualname(type(self)), type(arg))]
    return method(self, arg)


# The actual @visitor decorator
def visitor(arg_type):
    """Decorator that creates a visitor method."""

    def decorator(fn):
        declaring_class = _declaring_class(fn)
        _methods[(declaring_class, arg_type)] = fn

        # Replace all decorated methods with _visitor_impl
        return _visitor_impl

    return decorator

In [51]:
class Expression:
    def accept(self, visitor):
        visitor.visit(self)    


class DoubleExpression(Expression):
    def __init__(self, value):
        self.value = value


class AdditionExpression(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class ExpressionPrinter:
    def __init__(self):
        self.buffer = []

    @visitor(DoubleExpression)
    def visit(self, de):
        self.buffer.append(str(de.value))

    @visitor(AdditionExpression)
    def visit(self, ae):
        self.buffer.append('(')
        ae.left.accept(self) # This calls the accept of the correct type, which calls the visitor!
        self.buffer.append('+')
        ae.right.accept(self)
        self.buffer.append(')') # This calls the accept of the correct type, which calls the visitor!

    def __str__(self):
        return ''.join(self.buffer)

In [52]:
# __main__
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
buffer = []
printer = ExpressionPrinter()
printer.visit(e)
print(printer)

(1+(2+3))


### Classic Visitor Refined

The `accept()` method is necessary only for statically typed languages, i.e., languages in which viariable types are known at compiple time (e.g., C/C++). However, it is not necessary in non-statically typed languages, such as Python.

In this version, the `accept()` method is ignored and instead, `self.visit()` is used ina new class `ExpressionEvaluator`.

In [54]:
# VISITOR CODE
# Example taken from 
# https://tavianator.com/the-visitor-pattern-in-python/

def _qualname(obj):
    """Get the fully-qualified name of an object (including module)."""
    return obj.__module__ + '.' + obj.__qualname__


def _declaring_class(obj):
    """Get the name of the class that declared an object."""
    name = _qualname(obj)
    return name[:name.rfind('.')]


# Stores the actual visitor methods
_methods = {}


# Delegating visitor implementation
def _visitor_impl(self, arg):
    """Actual visitor method implementation."""
    method = _methods[(_qualname(type(self)), type(arg))]
    return method(self, arg)


# The actual @visitor decorator
def visitor(arg_type):
    """Decorator that creates a visitor method."""

    def decorator(fn):
        declaring_class = _declaring_class(fn)
        _methods[(declaring_class, arg_type)] = fn

        # Replace all decorated methods with _visitor_impl
        return _visitor_impl

    return decorator

In [57]:
class Expression:
    def accept(self, visitor):
        visitor.visit(self)    


class DoubleExpression(Expression):
    def __init__(self, value):
        self.value = value


class AdditionExpression(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class ExpressionPrinter:
    def __init__(self):
        self.buffer = []

    @visitor(DoubleExpression)
    def visit(self, de):
        self.buffer.append(str(de.value))

    @visitor(AdditionExpression)
    def visit(self, ae):
        self.buffer.append('(')
        ae.left.accept(self)
        self.buffer.append('+')
        ae.right.accept(self)
        self.buffer.append(')')

    def __str__(self):
        return ''.join(self.buffer)


# This class is stateful = we need to safe the evaluated value
# otherwise when the visitor is called in the next line,
# thevalue evaluated until that point is lost.
# We can achieve that with a temporal variable
# which caches the values.
class ExpressionEvaluator:
    def __init__(self):
        self.value = None

    @visitor(DoubleExpression)
    def visit(self, de):
        self.value = de.value

    @visitor(AdditionExpression)
    def visit(self, ae):
        # ae.left.accept(self)
        self.visit(ae.left)
        temp = self.value
        # ae.right.accept(self)
        self.visit(ae.right)
        self.value += temp

In [58]:
# __main__
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
printer = ExpressionPrinter()
printer.visit(e)

evaluator = ExpressionEvaluator()
evaluator.visit(e)

print(f'{printer} = {evaluator.value}')


(1+(2+3)) = 6
